In [1]:
from datetime import datetime
import os
import platform

import pytz

num_cores = os.cpu_count()

threshold = float(0.1)
threshold_str = str(threshold).replace(".", "-")

dataset_files = os.path.abspath("resources/input/2_ave_alumno_1_TH")

csv_file = os.path.abspath("data/processed/10Train copia.csv")

output_path = os.path.abspath("resources/output_json/")

json_output_path = os.path.abspath("resources/output_json")
date_time = datetime.now(pytz.timezone("Europe/Madrid")).strftime("%Y-%m-%d_%H-%M")

name = date_time + "_" + threshold_str

folder_name = os.path.join(json_output_path, name)
os.makedirs(folder_name)


file_path = folder_name + os.sep + name + ".json"


detection_model = os.path.abspath("models/MegaDetector v4.1, 2020.04.27/md_v4.1.0.pb")

ai4eutils_path = os.path.abspath("data/external/ai4eutils")
CameraTraps_path = os.path.abspath("data/external/CameraTraps")
src_path = os.path.abspath("./src")

try:
    os.environ["PYTHONPATH"]
except KeyError:
    os.environ["PYTHONPATH"] = ""
if platform.system() == "Windows":
    os.environ["PYTHONPATH"] += ";" + ai4eutils_path
    os.environ["PYTHONPATH"] += ";" + CameraTraps_path
    os.environ["PYTHONPATH"] += ";" + src_path
else:
    os.environ["PYTHONPATH"] += ":" + ai4eutils_path
    os.environ["PYTHONPATH"] += ":" + CameraTraps_path
    os.environ["PYTHONPATH"] += ":" + src_path

In [2]:
!python ./data/external/CameraTraps/detection/run_tf_detector_batch.py "$detection_model" "$dataset_files" "$file_path" --recursive --threshold "$threshold" --ncore "$num_cores"

TensorFlow version: 2.10.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Metal device set to: Apple M3

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

2024-05-21 21:01:24.457282: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-21 21:01:24.457356: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
Is GPU available? tf.test.is_gpu_available: True
TensorFlow version: 2.10.0
2024-05-21 21:01:24.459281: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-21 21:01:24.459

In [11]:
import json
import src.data.Dataset as dt

with open(file_path, "r") as file:
    data = json.load(file)

dataset = dt.load_from_csv(csv_file)
dataset = dt.convert_csv_to_abstract(dataset, dataset_files)

file_name_list = dataset["file_name"].tolist()

detection_label = []
false_positive = []
for image in data["images"]:
    file = image["file"]
    is_file_in_dataset = file in dataset["file_name"].values
    if is_file_in_dataset:
        pos = file_name_list.index(file)
        label = 0
        for detection in image["detections"]:
            if detection["category"] != "0":
                label = 1
                break
        detection_label.append(label)
        false_positive.append(int(dataset["label"].iloc[pos] != label))
dataset["detection_label"] = detection_label
dataset["is_false_positive"] = false_positive
total_false_positive = dataset["is_false_positive"].sum()

El archivo /Users/carlos/Library/Mobile Documents/com~apple~CloudDocs/WORKSPACE/TFG-Cameratraps_Detection/data/processed/10Train copia.csv se ha abierto con éxito.
1
